In [1]:
import math
from copy import deepcopy
from collections import Counter
from io import StringIO
from operator import itemgetter, attrgetter

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

import pydot_ng as pydot
from IPython.display import Image
from tabulate import tabulate

from ipywidgets import IntProgress
from IPython.display import display
import time

# Data Processing

## Load Data

In [2]:
df = pd.read_csv('./speed-dating-experiment/Speed Dating Data.csv', encoding="ISO-8859-1")

## Analyze Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB


In [4]:
# Unparseable columns

unparseable_cols = df.select_dtypes(include=['object']).columns
unparseable_cols

Index(['field', 'undergra', 'mn_sat', 'tuition', 'from', 'zipcode', 'income',
       'career'],
      dtype='object')

In [5]:
# Number of null value of each columns

cols_to_null_nums = df.isnull().sum(axis=0).sort_values(ascending=False)
cols_to_null_nums[:10]

num_in_3    7710
numdat_3    6882
expnum      6578
sinc7_2     6423
amb7_2      6423
shar7_2     6404
intel7_2    6394
fun7_2      6394
attr7_2     6394
amb5_3      6362
dtype: int64

In [6]:
# Number of null value of each row

rows_to_null_nums = df.isnull().sum(axis=1).sort_values(ascending=False)
rows_to_null_nums[:10]

842    172
847    172
845    172
839    172
843    172
838    171
324    158
841    155
846    154
840    154
dtype: int64

In [7]:
df.describe()

/Users/nacat/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,...,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
mean,283.675937,8.960248,0.500597,17.327166,1.828837,11.350919,16.872046,9.042731,9.295775,8.927668,...,7.240312,8.093357,8.388777,7.658782,7.391545,6.810020,7.615079,7.932540,7.155258,7.048611
std,158.583367,5.491329,0.500029,10.940735,0.376673,5.995903,4.358458,5.514939,5.650199,5.477009,...,1.576596,1.610309,1.459094,1.744670,1.961417,1.507341,1.504551,1.340868,1.672787,1.717988
min,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,4.000000,1.000000,1.000000
25%,154.000000,NaN,0.000000,8.000000,2.000000,7.000000,14.000000,4.000000,NaN,4.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,281.000000,NaN,1.000000,16.000000,2.000000,11.000000,18.000000,8.000000,NaN,8.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,407.000000,NaN,1.000000,26.000000,2.000000,15.000000,20.000000,13.000000,NaN,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,552.000000,22.000000,1.000000,44.000000,2.000000,21.000000,22.000000,22.000000,22.000000,22.000000,...,12.000000,12.000000,12.000000,12.000000,12.000000,10.000000,10.000000,10.000000,10.000000,10.000000


## Data Cleaning

In [8]:
# Clean Comma
COLUMNS_WITH_COMMA = ['income', 'zipcode', 'tuition', 'mn_sat']

for col in COLUMNS_WITH_COMMA:
    df[col] = df[col].replace('[,]', '', regex=True).astype(float)

### Handle String Data

In [9]:
ENCODED_STRING_COLUMNS = ['career', 'field']
df.drop(ENCODED_STRING_COLUMNS, axis=1, inplace=True)

In [10]:
OTHER_STRING_COLUMNS = ['undergra', 'from']

# Joined df based on partner
joined_df = df.merge(
    df,
    how='left',
    left_on=['iid', 'id', 'pid', 'partner'],
    right_on=['pid', 'partner', 'iid', 'id']
)

In [11]:
df['same_from'] = joined_df['from_x'] == joined_df['from_y']

In [12]:
df['same_undergra'] = joined_df['undergra_x'] == joined_df['undergra_y']

In [13]:
df.drop(OTHER_STRING_COLUMNS, axis=1, inplace=True)

---

# Method Implementation

## KNN

### Self Implement

In [14]:
def eucldn_similarity(v1, v2):
    """
    Euclidean Distance。越相似，距離越近，相似度數值會越小。
    :param v1:
    :param v2:
    :return: 1/distance(v1,v2) 取距離的倒數，越大越像
    """
    sum_xx, sum_xy, sum_yy = 0.0, 0.0, 0.0
    for item1, item2 in zip(v1, v2):
        sum_xx += math.pow(item1, 2)
        sum_xy += item1*item2
        sum_yy += math.pow(item2, 2)
    return 1 / math.sqrt(sum_xx + sum_yy - 2*sum_xy)

In [15]:
def cos_similarity(v1, v2):
    """
    Cosine（兩向量的餘弦）。越相似，夾角越小，相似度數值會越高。
    :param v1:
    :param v2:
    :return: cos(v1,v2)介於1到-1之間 越大越像
    """
    sum_xx, sum_xy, sum_yy = 0.0, 0.0, 0.0
    for item1, item2 in zip(v1, v2):
        sum_xx += math.pow(item1, 2)
        sum_xy += item1*item2
        sum_yy += math.pow(item2, 2)
    return sum_xy / math.sqrt(sum_xx*sum_yy)

In [16]:
def knn_prediction(nb_list):
    cnt = Counter(target for (sim, target) in nb_list)
    label = cnt.most_common(1)[0][0]
    return label

In [17]:
def knn_classify(test_X, train_X, train_y, k=5):
    nb = []
    for feature, label in zip(train_X, train_y):
        nb.append((cos_similarity(test_X,feature),tuple(label)))
    nb.sort(reverse=True)
    label = knn_prediction(nb[:k])
    return label

In [18]:
def knn(train_X, test_X, train_y, test_y, n_neighbors=5):
    count = 0
    for feature, label in zip(test_X, test_y):
        pred_y = knn_classify(feature, train_X, train_y, n_neighbors)
        if pred_y == tuple(label):
            count += 1
    accuracy = count / len(test_X)
    return accuracy

## Sklearn

In [21]:
def knn_with_sklearn(train_X, test_X, train_y, test_y,
                     n_neighbors=5):
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    return accuracy_score(pred, test_y)

# Decision Tree

## Self Implement

In [22]:
def gini_index(lcnt, rcnt):
    gini = 0.0
    tcnt = lcnt + rcnt
    tl = sum(lcnt.values())
    tr = sum(rcnt.values())
    tt = tl + tr
    l = 0.0
    r = 0.0
    for class_label in set(tcnt):
        if tl is not 0:
            l += pow(lcnt[class_label]/tl,2)
        if tr is not 0:    
            r += pow(rcnt[class_label]/tr,2)
    if tt is not 0:
        gini = (tl/tt)*(1-l)+(tr/tt)*(1-r)
    return gini

In [23]:
def try_split(train_X, train_y, index, value):
    gini = 1.0
    left_X, right_X = list(), list()
    left_y, right_y = list(), list()
    lcnt, rcnt, tcnt = Counter(), Counter(), Counter
    for row_X, row_y in zip(train_X, train_y):
        if row_X[index] < value:
            left_X.append(row_X)
            left_y.append(row_y)
            lcnt[tuple(row_y)] += 1
        else:
            right_X.append(row_X)
            right_y.append(row_y)
            rcnt[tuple(row_y)] += 1
    if not left_X:
        l = 0
    else:
        l = lcnt.most_common(1)[0][0]
    if not right_X:
        r = 0
    else:
        r = rcnt.most_common(1)[0][0]
    gini = gini_index(lcnt, rcnt)
    return gini, left_X, right_X, left_y, right_y, l, r

In [24]:
def determine_split(train_X, train_y):
    b_feature, b_value, b_gini, b_group = 999, 999, 999, None
    for index in range(len(train_X[0])):
        for tp in train_X:
            gini, left_group, right_group, left_label, right_label, l, f= try_split(train_X, train_y, index, tp[index])
            groups = left_group, right_group
            labels = left_label, right_label
            counter = l, f
            if gini < b_gini:
                b_index, b_value, b_gini, b_group, b_label, b_cnt= index, tp[index], gini, groups, labels, counter
    return {'index':b_index, 'value':b_value, 'gini':b_gini, 'groups':b_group, 'labels':b_label, 'count':b_cnt}

In [26]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    l_label, r_label = node['labels']
    l_cnt, r_cnt = node['count']
    #del(node['groups'])
    #del(node['labels'])

    if not left or not right:
        if isinstance(l_cnt, tuple):
            node['left'] = node['right'] = l_cnt
        elif isinstance(r_cnt, tuple):
            node['left'] = node['right'] = r_cnt
        else:
            node['left'] = node['right'] = None
        #node['left'] = node['right'] = countLabel(l_label+r_label)
        return
    if depth >= max_depth:
        node['left'], node['right'] = l_cnt, r_cnt#countLabel(l_label), countLabel(r_label)
        return
    if len(left) <= min_size:
        node['left'] = l_cnt#countLabel(l_label)
    else:
        node['left'] = determine_split(left, l_label)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = r_cnt#countLabel(r_label)
    else:
        node['right'] = determine_split(right, r_label)
        split(node['right'], max_depth, min_size, depth+1)

In [27]:
def build_tree(train_X, train_y, max_depth, min_size):
    root = determine_split(train_X, train_y)
    split(root, max_depth, min_size, 1)
    return root

In [28]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [29]:
def dt_predict(tp, node):
    if tp[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return dt_predict(tp, node['left'])
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return dt_predict(tp, node['right'])
        else:
            return node['right']

In [30]:
def dt_accuracy(test_X, test_y, tree):
    accuracy = 0
    count = 0
    for tp, label in zip(test_X, test_y):
        pred = dt_predict(tp, tree)
        if pred == tuple(label):
            count += 1
    accuracy = count/len(test_y)
    return accuracy

In [31]:
def dt(train_X, test_X, train_y, test_y, max_depth=5, min_samples_leaf=5):
    accuracy = 0
    tree = build_tree(train_X, train_y, max_depth, min_samples_leaf)
    print_tree(tree)
    accuracy = dt_accuracy(test_X, test_y, tree)
    return accuracy

## Sklearn

In [ ]:
def dt_with_sklearn(train_X, test_X, train_y, test_y,
                    min_samples_leaf=5):
    clf = tree.DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    return accuracy_score(pred, test_y)

In [ ]:
# Visualize Decision Tree

# dot_data = StringIO()
# tree.export_graphviz(clf, out_file=dot_data,
#                      feature_names=df_X.columns, class_names=True)
# graph = pydot.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

## Split Data Method

In [34]:
LABELS = ['dec', 'dec_o']
REDUDANT_COLUMNS = ['match']


def split_df(df,
             default_na=0, test_size=0.2, random_state=1):
    df = df.fillna(default_na).drop(REDUDANT_COLUMNS, axis=1)
    
    df_X = df.drop(LABELS, axis=1)
    df_y = df[LABELS]
    
    X, y = np.array(df_X), np.array(df_y)
    return train_test_split(X, y,
                            test_size=test_size, random_state=random_state)    

## Training Method

In [ ]:
def train_with_all_methods(df):
    data = split_df(df)
    print('Sklearn KNN Accurary: ', knn_with_sklearn(*data))
    print('Sklearn DT Accurary: ', dt_with_sklearn(*data))
    print('KNN Accurary: ', knn(*data))
    print('DT Accurary: ', dt(*data))

---

# Baseline

In [36]:
baseline_df = deepcopy(df)

## Data Processing

## Training

In [ ]:
train_with_all_methods(baseline_df)

---

# Drop columns or rows with too many null

In [ ]:
df_drop_nan = deepcopy(df)

## Data Processing

In [ ]:
# Drop Rows

ROW_THREASHOLD = 100

df_drop_nan.dropna(axis=0, thresh=ROW_THREASHOLD, inplace=True);

In [ ]:
# Drop Cols

COL_THREASHOLD = 4000

df_drop_nan.dropna(axis=1, thresh=COL_THREASHOLD, inplace=True);

## Training

In [ ]:
train_with_all_methods(df_drop_nan)

---

# Hobby Similarity

In [ ]:
df_hs = deepcopy(df)

## Data Processing

In [ ]:
HOBBY_COLUMNS = [
    'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art',
    'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater',
    'movies', 'concerts', 'music', 'shopping', 'yoga'
]

similiar_thres = 1

In [ ]:
def count_hobby_similarity(joined_df,
                           similiar_thres=1,
                           na_default=-1):
    joined_df_no_na = joined_df.fillna(na_default)
    ss = list()
    for hobby in HOBBY_COLUMNS:
        temp_s = abs(joined_df_no_na[hobby+'_x'] - joined_df_no_na[hobby+'_y'])
        temp_s = (temp_s <= similiar_thres).astype(int)
        ss.append(temp_s)
    return sum(ss)

In [ ]:
df_hs['sim_hob_num'] = count_hobby_similarity(joined_df, similiar_thres=1)

In [ ]:
df_hs.drop(HOBBY_COLUMNS, axis=1, inplace=True)

## Training

In [ ]:
train_with_all_methods(df_hs)